In [1]:
from qat.core import Term
from qat.fermion.hamiltonians import FermionHamiltonian
from qat.lang import *
from qat.qpus import get_default_qpu
from qat.plugins import ScipyMinimizePlugin
import numpy as np

using legacy validation callback
c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


On commence d'abord par définir l'opérateur $c^{\dagger}_{i, \sigma1}c_{j, \sigma2}$ qui va être utilisé pour définir le hamiltonien :

In [ ]:
n_qubits = int(input())

#Definition de l'operateur c^{dague}_{i, \sigma1}c{j, \sigma2} 
def couplingOp(energy, i, j, spin_i, spin_j) : #i est le numéro du site
    if spin_i == "+" :
         if spin_j == "-" :
            return FermionHamiltonian(n_qubits, [Term(energy, "Cc", [i, j+1])])
         else :
             return FermionHamiltonian(n_qubits, [Term(energy, "Cc", [i, j])])
    else :
        if spin_j == "-" :
            return FermionHamiltonian(n_qubits, [Term(energy, "Cc", [i+1, j+1])])
        else :
             return FermionHamiltonian(n_qubits, [Term(energy, "Cc", [i+1, j])]) 
        
def nOp(energy, i, spin) :
    return couplingOp(energy, i, i, spin, spin)
    

Definition de l'Hamiltonien

In [ ]:
#Definition de l'hamiltonien
def hamiltonian(n_impurities, n_qubits, mu, t, U, V, epsilon) :
    H_repulsion = 0
    H_hopping = 0
    H_chem = 0
    for wire in range(0, 2*n_impurities, 2) :
        #Calcul du terme d'origine chimique
        H_chem += nOp(-mu, wire, "+")
        H_chem += nOp(-mu, wire, "-")

        #Calcul du terme d'effet tunnel entre impuretés
        for wire2 in range(0, 2*n_impurities, 2) :
            if wire2 != wire :
                H_hopping += couplingOp(-t, wire, wire2, "+", "-")
                H_hopping += couplingOp(-t, wire, wire2, "+", "+")
                H_hopping += couplingOp(-t, wire, wire2, "-", "-")
                H_hopping += couplingOp(-t, wire, wire2, "-", "+")
            else :
                #Calcul du terme de répulsion
                H_repulsion += nOp(U, wire, "+")*nOp(1, wire,"-")
    
    H_impurity_bath = 0
    H_bath = 0
    for wire3 in range(2*n_impurities, n_qubits, 2) :
        i = (wire3 - 2*n_impurities) // 2
        print(i)
        e = epsilon[i]
        #Calcul du terme lié au bain
        H_bath += nOp(e, wire3, "+")
        H_bath += nOp(e, wire3, "-")

        for wire4 in range(0, 2*n_impurities, 2) :
            #Calcul du terme d'interaction impureté-bain
            j = wire4 // 2
            v = V[j][i]
            v_conj = v.conjugate()
            H_impurity_bath += couplingOp(v, wire4, wire3, "+", "-")
            H_impurity_bath += couplingOp(v_conj, wire3, wire4, "+", "-")

            H_impurity_bath += couplingOp(v, wire4, wire3, "+", "+")
            H_impurity_bath += couplingOp(v_conj, wire3, wire4, "+", "+")

            H_impurity_bath += couplingOp(v, wire4, wire3, "-", "-")
            H_impurity_bath += couplingOp(v_conj, wire3, wire4, "-", "-")

            H_impurity_bath += couplingOp(v, wire4, wire3, "-", "+")
            H_impurity_bath += couplingOp(v_conj, wire3, wire4, "-", "+")
        
    H_anderson = H_repulsion + H_chem + H_impurity_bath + H_bath + H_hopping
    H_spin = H_anderson.to_spin(method = "jordan-wigner")#Fermion to Qubit
    return H_spin, H_spin.get_matrix()

AndersonHamiltonian, matrix = hamiltonian(1, 4, 1, 1, 1, [[1]], [0.5]) 
print(AndersonHamiltonian)

0
(-0.25+0j) * I^6 +
(0.25+0j) * (ZZ|[0, 1]) +
(0.25+0j) * (Z|[0]) +
(0.25+0j) * (Z|[1]) +
(0.5+0j) * (XZX|[0, 1, 2]) +
(0.5+0j) * (YZY|[0, 1, 2]) +
(0.5+0j) * (XZX|[1, 2, 3]) +
(0.5+0j) * (YZY|[1, 2, 3]) +
(0.5+0j) * (YY|[1, 2]) +
(0.5+0j) * (XX|[1, 2]) +
(0.5+0j) * (XZZX|[0, 1, 2, 3]) +
(0.5+0j) * (YZZY|[0, 1, 2, 3]) +
(-0.25+0j) * (Z|[2]) +
(-0.25+0j) * (Z|[3])


Conception du circuit :

In [1]:
@qrout
def circuit(params) :
    

SyntaxError: incomplete input (2202165594.py, line 3)